In [10]:
library(xgboost)
library(readr)
library(stringr)
library(caret)
library(car)

Warning message:
“package ‘car’ was built under R version 3.4.3”

In [11]:
#### Obtention des données 
data_expressions <- read.csv("data/expressions_train.txt",sep = " ")

X_expressions <- data_expressions[,1:4200]
y_expressions <-data_expressions$y

table(y_expressions)

y_expressions
   anger  disgust     fear      joy  sadness surprise 
      17       15       16       21       23       16 

In [12]:
I<-matrix(as.matrix(X_expressions[14,]),60,70)
I1 <- apply(I, 1, rev)
## Uncomment to plot image 
#image(t(I1),col=gray(0:255 / 255))
#image(matrix(unlist(I1),ncol=70, byrow =TRUE ),col=gray(0:255 / 255))

In [13]:
dim(X_expressions)
dim(X_expressions[complete.cases(X_expressions), ])
dim(na.omit(X_expressions))
tst <-na.omit(unname(X_expressions))

[1]  108 4200

[1]  108 4200

[1]  108 4200

In [14]:
#### Separation train-test

n=nrow(data_expressions)
ntrain=ceiling(n*2/3)
ntst=n-ntrain
train<-sample(1:n,ntrain)

data_expressions.test<-data_expressions[-train,]
data_expressions.train<-data_expressions[train,]

In [15]:
# Except the y vector, all the feature are numerical so we do not need any transformation there
#sparse_matrix <- sparse.model.matrix(response ~ .-1, data = campaign)
df_X<- X_expressions
sparse_matrix <- sparse.model.matrix(response ~ .-1, data = campaign)
output_vector = df[,response] == "Responder"

ERROR: Error in sparse.model.matrix(response ~ . - 1, data = campaign): could not find function "sparse.model.matrix"
